In [ ]:
import pymysql
import json
import re
from typing import List, Dict

def extract_doc_id(source_id: str) -> str:
    """sourceId에서 중간 숫자 추출 (예: doc_22677031_02 -> 22677031)"""
    match = re.search(r'doc_(\d+)_\d+', source_id)
    return match.group(1) if match else None

def get_reference_documents(cursor, source_doc_id: str) -> List[str]:
    """DB에서 해당 source_document_id의 모든 reference_document_id 조회"""
    query = """
        SELECT reference_document_id 
        FROM reference_documents 
        WHERE source_document_id = %s
    """
    cursor.execute(query, (source_doc_id,))
    results = cursor.fetchall()
    return [str(row['reference_document_id']) for row in results]

def update_cmds_references(cmds_file_path: str, output_file_path: str = None):
    """cmds 파일의 references를 DB 데이터로 업데이트"""
    
    # 출력 파일 경로 설정
    if output_file_path is None:
        output_file_path = cmds_file_path.replace('.cmds', '_updated.cmds')
    
    # DB 연결
    connection = pymysql.connect(
        host='localhost',
        user='root',  # 사용자명 수정
        password='1234',  # 비밀번호 수정
        database='any_approval',
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
    
    try:
        with connection.cursor() as cursor:
            # cmds 파일 읽기
            with open(cmds_file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
            
            updated_lines = []
            update_count = 0
            no_reference_count = 0
            
            for line in lines:
                line = line.strip()
                
                # 빈 줄은 그대로 유지
                if not line:
                    updated_lines.append('')
                    continue
                
                # addDocument로 시작하는 줄 처리
                if line.startswith('addDocument '):
                    try:
                        # JSON 파싱
                        json_str = line.replace('addDocument ', '', 1)
                        doc_data = json.loads(json_str)
                        
                        # sourceId에서 문서 ID 추출
                        source_id = doc_data.get('sourceId', '')
                        doc_id = extract_doc_id(source_id)
                        
                        if doc_id:
                            # DB에서 참조 문서 조회
                            reference_ids = get_reference_documents(cursor, doc_id)
                            
                            # references 업데이트
                            if reference_ids:
                                doc_data['references'] = [
                                    {"sourceId": f"doc_{ref_id}_02"} 
                                    for ref_id in reference_ids
                                ]
                                update_count += 1
                                print(f"✓ {source_id}: {len(reference_ids)}개의 참조 문서 업데이트")
                            else:
                                doc_data['references'] = []
                                no_reference_count += 1
                                print(f"○ {source_id}: 참조 문서 없음")
                        else:
                            print(f"✗ sourceId 파싱 실패: {source_id}")
                        
                        # 업데이트된 JSON을 다시 문자열로
                        updated_line = 'addDocument ' + json.dumps(doc_data, ensure_ascii=False)
                        updated_lines.append(updated_line)
                        
                    except json.JSONDecodeError as e:
                        print(f"✗ JSON 파싱 오류: {e}")
                        updated_lines.append(line)
                    except Exception as e:
                        print(f"✗ 처리 중 오류: {e}")
                        updated_lines.append(line)
                else:
                    # addDocument가 아닌 줄은 그대로 유지
                    updated_lines.append(line)
            
            # 업데이트된 내용을 파일에 저장
            with open(output_file_path, 'w', encoding='utf-8') as f:
                for line in updated_lines:
                    f.write(line + '\n')
            
            print(f"\n{'='*60}")
            print(f"작업 완료!")
            print(f"- 총 처리: {len([l for l in updated_lines if l.startswith('addDocument')])}개 문서")
            print(f"- references 업데이트: {update_count}개")
            print(f"- 참조 문서 없음: {no_reference_count}개")
            print(f"- 저장 경로: {output_file_path}")
            print(f"{'='*60}")
            
    finally:
        connection.close()

# 실행
if __name__ == "__main__":
    # cmds 파일 경로 지정
    cmds_file_path = r"C:\Users\LEEJUHWAN\Downloads\documents_2015_imgFixed.cmds" # 실제 파일 경로로 수정
    
    # references 업데이트 실행
    update_cmds_references(cmds_file_path)